In [16]:
import requests
import time
import pandas as pd
import glob
import json

## STEAM Game ID 불러오기

In [10]:
def get_game_app_id(game_url):
    app_id = game_url.split('/')[-3]

    return app_id

In [11]:
game_url = 'https://store.steampowered.com/app/477160/Human_Fall_Flat/'
get_game_app_id(game_url)

'477160'

## OpenAPI를 통한 데이터 수집 및 살펴보기

In [12]:
def get_steam_reviews(app_id, num_per_page=100, delay=1.0):
    cursor = '*'  # 첫 요청에서는 '*'를 사용하거나 생략 가능
    reviews = []

    while True:
        url = f"https://store.steampowered.com/appreviews/{app_id}?json=1"
        params = {
            'num_per_page': num_per_page,  # 한 번의 요청으로 가져올 수 있는 리뷰의 최대 개수
            'cursor': cursor,  # 페이지 네이션 처리를 위함
            'filter': 'updated',  # 최신 리뷰부터 가져오도록 설정
            'language': 'koreana',  # 한국어 리뷰만 가져옴
            'review_type': 'all',
            'purchase_type': 'all',
        }

        try:
            response = requests.get(url, params=params, timeout=10)  # 타임아웃 10초 설정
            data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break

        if data.get('success') != 1:
            print("API 호출 실패")
            break

        new_reviews = data.get('reviews', [])
        reviews.extend(new_reviews)  # 리뷰 데이터 추가

        # 다음 페이지로 이동하기 위한 cursor 업데이트
        cursor = data.get('cursor')

        # 리뷰가 더 이상 없거나 불러온 리뷰 개수가 적으면 중단
        if not cursor or len(new_reviews) < num_per_page:
            break

        # 너무 빠르게 요청하면 서버에서 차단될 수 있으니, 지연시간 추가
        time.sleep(delay)

    return reviews

In [9]:
all_reviews = get_steam_reviews(get_game_app_id(game_url))

# 리뷰 개수 출력
print(f"총 리뷰 개수: {len(all_reviews)}")

총 리뷰 개수: 10014


In [10]:
print(all_reviews[1])

{'recommendationid': '178417609', 'author': {'steamid': '76561198195367613', 'num_games_owned': 0, 'num_reviews': 23, 'playtime_forever': 3444, 'playtime_last_two_weeks': 0, 'playtime_at_review': 3444, 'last_played': 1727765993}, 'language': 'koreana', 'review': 'Good game', 'timestamp_created': 1730768512, 'timestamp_updated': 1730768512, 'voted_up': True, 'votes_up': 0, 'votes_funny': 0, 'weighted_vote_score': 0, 'comment_count': 0, 'steam_purchase': False, 'received_for_free': False, 'written_during_early_access': False, 'primarily_steam_deck': False}


### all_reviews 주요 key-value 설명
1. **`recommendationid`**: 리뷰의 고유 ID. 
2. **`author`**: 리뷰 작성자에 관한 정보
     - **`steamid`**: 작성자의 고유 Steam ID
     - **`num_games_owned`**: 작성자가 소유한 총 게임 수
     - **`num_reviews`**: 작성자가 작성한 전체 리뷰 수
     - **`playtime_forever`**: 작성자가 이 게임에서 누적한 총 플레이 시간(분 단위)
     - **`playtime_last_two_weeks`**: 최근 2주 동안의 플레이 시간(분 단위)
     - **`playtime_at_review`**: 이 리뷰를 작성할 당시의 플레이 시간(분 단위)
     - **`last_played`**: 작성자가 마지막으로 이 게임을 플레이한 시점을 유닉스 타임스탬프 형식으로 표현
3. **`language`**: 리뷰가 작성된 언어.  **`koreana`**=한국어
4. **`review`**: 작성자가 남긴 리뷰 텍스트입니다.
5. **`timestamp_created`**: 리뷰가 처음 작성된 시점, 유닉스 타임스탬프 형식
6. **`timestamp_updated`**: 리뷰가 마지막으로 수정된 시점, 유닉스 타임스탬프. 처음 작성 이후 업데이트가 있을 때 업데이트 시간을 기록
7. **`voted_up`**: 작성자가 이 게임을 추천했는지 여부를 나타내며, `True`일 경우 추천, `False`일 경우 추천하지 않음
8. **`votes_up`**: 다른 사용자들이 이 리뷰를 '유용함'으로 평가한 수
9. **`votes_funny`**: 다른 사용자들이 이 리뷰를 '재미있음'으로 평가한 수
10. **`weighted_vote_score`**: 리뷰의 가중치가 부여된 평가 점수입니다. 이 값은 유용함, 재미 등 사용자 피드백에 기반하여 가중치가 적용된 점수로 계산됨
11. **`comment_count`**: 이 리뷰에 대해 다른 사용자가 남긴 댓글 수
12. **`steam_purchase`**: 리뷰 작성자가 게임을 직접 구매한 후 작성한 리뷰인지 여부. `True`이면 직접 구매 후 작성한 리뷰, `False`이면 아닌 경우.
13. **`received_for_free`**: 리뷰 작성자가 게임을 무료로 제공받았는지 여부. `True`일 경우 무료로 받았고, `False`일 경우 직접 구매했음을 의미.
14. **`written_during_early_access`**: 이 리뷰가 게임의 `얼리 액세스` 기간 동안 작성되었는지 여부. `True`이면 얼리 액세스 중 작성된 리뷰임을 의미.
15. **`hidden_in_steam_china`**: 이 리뷰가 Steam의 중국 버전에서 숨겨져 있는지 여부. `True`일 경우 중국 버전에서 이 리뷰가 보이지 않음.
16. **`steam_china_location`**: 중국 Steam 사용자에 대한 지역 정보를 나타내는 값이나, 빈 문자열일 경우 지역 정보가 없음을 의미.
17. **`primarily_steam_deck`**: 이 리뷰가 Steam Deck 기기에서 주로 작성되었는지 여부. `True`이면 Steam Deck에서 작성된 리뷰.

## positive/negative data 차이
- `voted_up` key 값이 True면 positive, False면 negative임

In [13]:
game_url = [
    'https://store.steampowered.com/app/1085660/_/', # Destiny 2
    'https://store.steampowered.com/app/2357570/_2/', # Overwatch 2
    'https://store.steampowered.com/app/1225570/Unravel_Two/', # Unravel Two
    'https://store.steampowered.com/app/108600/Project_Zomboid/', # Project Zomboid
    'https://store.steampowered.com/app/381210/Dead_by_Daylight/', # Dead by Daylight
    'https://store.steampowered.com/app/578080/PUBG_BATTLEGROUNDS/', # Battlegrounds
    'https://store.steampowered.com/app/2669320/EA_SPORTS_FC_25/', # EA_SPORTS_FC_25
    'https://store.steampowered.com/app/952060/BIOHAZARD_RE3/', # Resident Evil 3
    'https://store.steampowered.com/app/275850/No_Mans_Sky/', # No Mans Sky
    'https://store.steampowered.com/app/648800/Raft/' # Raft
]

In [14]:
app_ids = [get_game_app_id(url) for url in game_url]
game_titles = ['Destiny 2', 'Overwatch 2', 'Unravel Two', 'Project Zomboid', 'Dead by Daylight', 'Battlegrounds', 'EA_SPORTS_FC_25', 'Resident Evil 3', 'No Mans Sky', 'Raft']

In [9]:
for i, v in enumerate(app_ids):
    reviews = get_steam_reviews(v)
    print(len(reviews))
    df = pd.DataFrame(reviews)
    df.to_csv(f'./csv_files/steam_reviews_{game_titles[i]}.csv', index=False)

14062
3124
159
9239
21863
6299
99
1834
2449
9346


In [ ]:
for title in game_titles:
    df = pd.read_csv(f'./csv_files/steam_reviews_{title}.csv')
    
    # author 컬럼을 ','를 기준으로 나누기
    df['author'] = df['author'].apply(lambda x: json.loads(x.replace("'", '"')))
    
    author_df = pd.json_normalize(df['author'])

    # author 컬럼 삭제
    df = pd.concat([df.drop(columns=['author']), author_df], axis=1)
    
    df.to_csv(f'./csv_files/steam_reviews_{game_titles[i]}_flatten.csv', index=False)

In [ ]:
csv_files = glob.glob(f"./csv_files/steam_reviews_*_flatten.csv")

combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

combined_df.to_csv('./output.csv', index=False)

In [18]:
df = pd.read_csv(f'./csv_files/steam_reviews_HFF.csv')
    
# author 컬럼을 ','를 기준으로 나누기
df['author'] = df['author'].apply(lambda x: json.loads(x.replace("'", '"')))

author_df = pd.json_normalize(df['author'])

# author 컬럼 삭제
df = pd.concat([df.drop(columns=['author']), author_df], axis=1)

df.to_parquet(f'./csv_files/steam_reviews_HFF_flatten.parquet', index=False)